<a href="https://colab.research.google.com/github/reganmeloche/mrpc_paraphrase/blob/main/semantic_similarity_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Paraphrase Detection - Semantic Similarity Approach


In this notebook, we will be following the approach laid out in the following paper: https://ieeexplore.ieee.org/document/8639211


## Import Data

In [ ]:
import pandas as pd
import csv

In [ ]:
ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/NLP/ms_paraphrase'

In [ ]:
data_path = f'{ROOT_PATH}/data'

train_df = pd.read_csv(f'{data_path}/train_df.csv')
test_df = pd.read_csv(f'{data_path}/test_df.csv')

In [ ]:
test_df.head()

## Preprocessing

We will perform some basic preprocessing and normalization of the sentences using the nltk library

In [ ]:
from string import punctuation
import numpy as np

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))

In [ ]:
def preprocess(text):
    # Lowercase
    text = text.lower()

    # Remove dashes
    text = text.replace("-", " ")

    # Remove punctuation
    text = ''.join(c for c in text if c not in punctuation)

    # Tokens
    tokens = [word for word in nltk.word_tokenize(text)]

    # Remove stop words
    non_sw = [t for t in tokens if t not in stop_words]
    text = ' '.join(t for t in non_sw)

    return text

In [ ]:
sample_text = 'I took the 3 dogs for a walk last Tuesday at 8pm!'
sample_prepro = preprocess(sample_text)
print(sample_prepro)

Next we will write a function that will format our input dataframe by iterating over the preprocess function. The result is a tuple, containing the sentence pairs (in another tuple) and the corresponding labels (1 or 0)

In [ ]:
def format_df(df):
    s1_list = df['s1'].apply(preprocess).values
    s2_list = df['s2'].apply(preprocess).values
    labels = df['label'].values
    sentence_pairs = list(zip(s1_list, s2_list))
    return sentence_pairs, labels

In [ ]:
X, y = format_df(train_df) 

print(X[0])

## Word Vectors

Now that our text is preprocessed, we want to extract word vectors. In our baseline approach, we used simple TFIDF vectors, which was based on counts of the words in the training set. In this approach we are going to use word embeddings, which are learned vector representations from a larger training corpus of generic text.

We are going to make use of the gensim library, which contains a variety of word embeddings that differ in vector dimension and also training corpus. We could potentially experiment with many of these and compare results. For now we will choose a fairly robust one: glove-wiki-gigaword-300. This can take a while to download.

Once it is downloaded, we can look up a word to see the 300-dimensional vector embedding of that word. Some more obscure words may not appear in the embedding. We will discuss how to handle those next.

In [ ]:
import gensim.downloader

print(list(gensim.downloader.info()['models'].keys()))


In [ ]:
embedding = gensim.downloader.load('glove-wiki-gigaword-300')

In [ ]:
embedding.get_vector('turkey')

One consideration that we must address is how we handle out-of-vocabulary (OOV) words, i.e. words that appear in our data but not in the training corpus. 

Our strategy is as follows: Every time we encounter an OOV word, we will generate a random 300-dimensional vector of values between -1 and 1. We will keep a dictionary of these random vectors in storage so that if we encounter that OOV word *again*, then it will have a vector representation.

We will now define a class that lets us get the word vectors for all the words in a given sentence. We simply look up each word in our embedding vocabulary. If it is there, then return the vector. If not, then we generate our random vector and store it in the dictionary. The class also handles the dictionary of random OOV vectors. 

We will also add a function that allows us to calculate the cosine similarity for two words, which we will make use of shortly

In [ ]:
from scipy import spatial

class EmbeddingWrapper:
    def __init__(self, embedding, vector_dim=300):
        self.__rand_dict = {}
        self.__embedding = embedding
        self.__vector_dim = vector_dim
    
    def get_sentence_vec(self, sentence):
        result = []

        for w in sentence.split(' '):
            next_v = self.get_word_vec(w)
            result.append(next_v)
        
        return result
    

    def get_word_vec(self, w):
        result = None
        if w in self.__embedding.vocab:
            result = self.__embedding.get_vector(w) 
        else:
            if w not in self.__rand_dict:
                v = np.random.random(self.__vector_dim) * 2 - 1 # Generate a random vector
                self.__rand_dict[w] = v
            result = self.__rand_dict[w]
        
        return result
    

    def get_distance(self, w1, w2):
        e1 = self.get_word_vec(w1)
        e2 = self.get_word_vec(w2)
        result = 1 - spatial.distance.cosine(e1, e2)
        return round(result,3)




In [ ]:
ew = EmbeddingWrapper(embedding, 300)

In [ ]:
test_vec = ew.get_sentence_vec(X[0][0])

ew.get_distance('cat', 'dog')

## Similarity Calculations

Our paraphrase determination will consist of several steps. For a given sentence pair, we are going to calculate a combination of similarity metrics and then mesh them together.

The first similarity calculation tool we use will be the cosine similarity that we previously defined.

The second tool we use will be WordNet. WordNet allows us to enter a word to get its synset, which is a collection of semantic information about a word, including relationships to other words that have a similar meaning. The synset object has a path similarity function that we will leverage


In [ ]:
def get_wn_sim(w1, w2):
        try:
            # Could improve here - get the actual wordnet pos type and extract the proper synset, rather than just the first one
            a = wn.synsets(w1)[0]
            b = wn.synsets(w2)[0]
            result = a.path_similarity(b)
        except:
            #print(f'oops: {s1, s2}')
            result = 0

        return round(result, 3)

In [ ]:
test_wn_calc = get_wn_sim('cat', 'dog')
print(test_wn_calc)

Now given two words, we have two measurements we can apply - the wordnet similarity metric and the cosine distance on the word embeddings. Our goal is to check if two *sentences* are similar - not individual words. Our determination of sentence similarity will be based on the individual similarities of the words. In this way, we are ignoring the sequential nature of the text, which is a considerable weakness.

We will combine these two scores into a single score by applying a weighting. The cosine distance will get a weight of 0.75, and the wordnet similarity gets a weight of 0.25. We will call this the *joint similarity*. We will need to pass in our embedding wrapper as well.


In [ ]:
def calculate_joint_similarity(ew, s1, s2, do_print=False):
        results = []

        for w1 in s1.split(' '):
            res_row = []
            for w2 in s2.split(' '):
                a = ew.get_distance(w1, w2)
                b = get_wn_sim(w1, w2)
                next_res = round(0.75 * a + 0.25 * b, 3)
                if do_print: print(f'{w1}/{w2}: {a}, {b} -> {next_res}')
                res_row.append(next_res)

            results.append(max(res_row))

        if do_print: print(results)
        final_result = np.average(results)
        return round(final_result, 3)

Let's look at a quick example. Suppose we have two sentences:
- The bird in the bush
- The cat in the bag

Once these have been preprocessed, they are simplified to:
- bird bush
- cat bag

We want to measure how similar these sentences are. Let's look at the scores for each word pairing. Each pair shows the cosine distance followed by wordnet similarity, followed by the joint similarity
- bird/cat: 0.398, 0.143 -> 0.334 
- bird/bag: 0.118, 0.091 -> 0.111
- bush/cat: 0.051, 0.077 -> 0.057
- bush/bag: 0.1, 0.091 -> 0.098

For each word in the first sentence, we will take the maximum score from the words in the second sentence. So for the word "bird", we use 0.334, and for the word "bush", we use 0.098. Finally, we take an average of these values.


In [ ]:
s1 = 'bird bush'
s2 = 'cat bag'

ss1 = calculate_joint_similarity(ew, s1, s2, do_print=True)

print(ss1)

We will perform this calculation again, but will consider s2 as our first sentence. We can then take the average of these two results to get $S_a$

In [ ]:
ss2 = calculate_joint_similarity(ew, s2, s1, do_print=True)
print(ss2)

score1 = (ss1 + ss2) / 2

print(f'\nAveraged joint score: {score1}')

We are going to incorporate one more score into our system. We mentioned that we are ignoring the word order in our previous calculation, which is a weakness. This next scoring mechanism will account for the order by building a word order vector for each sentence.

Let's consider two sentences from the paper:
- s1: big mouse clobbers small cat
- s2: small cat clobbers big mouse

We first take all the unique words in our sentences and assign them an index: { 1-big, 2-mouse, 3-clobbers, 4-small, 5-cat }. Then we build a vector showing the word order index. 

For s1, this vector is v1 = [1,2,3,4,5]. For s2, it is v2 = [4,5,3,1,2]

Our score $S_b$ is calculated as follows:

$S_b = 1 - \frac{\lVert V_1 - V_2 \rVert}{\lVert V_1 + V_2 \rVert} $.

When terms match up between the two sentences (e.g. "clobbers"), then $V_1 - V_2$ will get a zero term. This is the numerator of the fraction, so it push the entire fraction lower. Since we're subtracting the fraction from 1, this will yield a higher result. Therefore this score rewards sentences that have alignment in word order between the same word.
 





In [ ]:
def calculate_word_order_similarity(s1, s2):
    a = s1.split(' ')
    b = s2.split(' ')
    all_words = np.concatenate((a,b))
    word_set = set(all_words)

    v1 = np.zeros(len(word_set))
    v2 = np.zeros(len(word_set))

    for i, w in enumerate(word_set):
        for j in range(len(a)):
            if w == a[j]:
                v1[i] = j+1 
        
        for k in range(len(b)):
            if w == b[k]:
                v2[i] = k+1
    
    nx1 = np.linalg.norm(v1-v2)
    nx2 = np.linalg.norm(v1+v2)

    return 1 - (nx1/nx2)

In [ ]:
s1 = 'big mouse clobbers small cat'
s2 = 'small cat clobbers big mouse'
sb = calculate_word_order_similarity(s1, s2)
print(sb)

## Putting it all together

Now we have all of our scoring pieces. Now we just need to put them all together and decide how we want to weight the different scores. We perform the weighting using the delta parameter, which is between 0 and 1. The final score calculation is as follows:

$S = (\delta \cdot S_a) + (1 - \delta) \cdot S_b$


A high delta will emphasize the averaged joint score $S_a$, and a low delta will emphasize the word order sscore $S_b$

We define a class to bring this calculation together. We add the following functions
- Predict: We will use this later once a threshold is determined for deciding if a pair of sentences match
- calculate_all: Given a set of sentence pairs, calcuate all their scores and return it as a list
- calculate_pair_similarity: Calculates the similarity score for a single pair

In [ ]:
class SimilarityCalculator:
    def __init__(self, embedding_wrapper, delta):
        self.__ew = embedding_wrapper
        self.__delta = delta
    
    def predict(self, X, threshold):
        y_probs = self.calculate_all(X)
        results = [1 if y_prob > threshold else 0 for y_prob in y_probs]
        return results


    def calculate_all(self, X):
        results = []

        for i,x in enumerate(X):
            #if i%50==0: print(i)
            next_res = self.calculate_pair_similarity(x[0], x[1])
            results.append(next_res)

        return results

        
    def calculate_pair_similarity(self, s1, s2):
        ss1 = calculate_joint_similarity(self.__ew, s1, s2)
        ss2 = calculate_joint_similarity(self.__ew, s2, s1)

        s_a = (ss1 + ss2) / 2
        s_b = calculate_word_order_similarity(s1, s2)

        return (self.__delta * s_a) + ((1 - self.__delta) * s_b)




In [ ]:
sc = SimilarityCalculator(ew, 0.6)

result = sc.calculate_pair_similarity('small cat clobbers big mouse', 'big mouse clobbers small cat')

print(result)

## Learning a threshold

Just like we did with the TFIDF approach, we will now use our training data to learn the ideal threshold. This training can take a while due to the use of some of our internal tools, such as calls to wordnet.

We only consider the pairs that are considered a match, and we take their average score to get our prediction threshold.

In [ ]:
distances = sc.calculate_all(X)


In [ ]:
# Filter to keep only those that are labeled as a paraphrase
matches = [d for i,d in enumerate(distances) if y[i] == 1]

# calculate the average
threshold = np.average(matches)

In [ ]:
threshold

## Test set and Evaluation

Now we can use the threshold value and make predictions on the test set. Finally we can evaluate our performance against the actual labels for the test


In [ ]:
threshold = 0.58

In [ ]:
X_test, y_test = format_df(test_df) 

In [ ]:
y_pred = sc.predict(X_test, threshold)

In [ ]:
from sklearn import metrics

print(metrics.classification_report(y_test, y_pred))